# Projet : Planifier mes prochaines vacances 

### Nettoyage et analyse des données que nous avons collectées sur les villes et les hôtels. À la fin, il faut les enregistrer dans un seau S3 de notre compte AWS.

## Import librairies

In [1]:
!pip install boto3
# !pip install plotly
# !jupyter labextension install jupyterlab-plotly

In [2]:
import pandas as pd
import glob
import plotly.express as px
import boto3

## 1. Lire le fichier `.csv` qui contient des informations sur les villes et la météo.

In [3]:
cities = pd.read_csv('res/1_destinations.csv')
cities.head()

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature,rank,inverted_rank
0,21,Aix en Provence,43.529842,5.447474,Clouds,0.2327,14.34500,1,35
1,22,Avignon,43.949249,4.805901,Rain,2.2042,15.93750,2,34
2,8,Strasbourg,48.584614,7.750713,Clouds,2.5916,12.34875,3,33
3,2,Bayeux,49.276462,-0.702474,Rain,2.8621,11.71000,4,32
4,20,Marseille,43.296174,5.369953,Clouds,3.5174,14.50125,5,31


## 2. Lire les fichiers `.json` contenant les informations sur les hôtels et les enregistrez dans un seul Dataframe pandas. 
### on va enregistrer dans ce  Dataframe ces informations:

#### - le nom de la ville
#### - id de la ville 
#### - créer une colonne contenant un identifiant unique de l'hôtel (hotel_id)
#### - le nom de l'hôtel
#### - l'url
#### - la latitude
#### - la longitude

In [4]:
# Récupérer tous les fichiers du dossier `res` en commençant par `2_hotels_` et 
# en finissant par `.json`

hotel_files = glob.glob('res/2_hotels_''.json')

# Créer un nouveau DataFrame

hotels = pd.DataFrame(columns = ['city_id', 'city_name', 'hotel_id', 'name', 'url', 'latitude', 'longitude', 'score', 'description'])

# Itérer sur tous les fichiers JSON

for f in hotel_files:
    city_name = f.split('_')[2].split('.')[0].replace("-"," ")
    city_id = cities.loc[cities['name'] == city_name,'city_id'].values[0]
    
    print("Processing {}".format(city_name))
    
    # Lire les fichiers json et ajouter hotel_id, city_id et city_name dans les DataFrames
    
    temp_dataset = pd.read_json(f)
    temp_dataset = temp_dataset.reset_index().rename({'index': 'hotel_id'}, axis = 1)
    temp_dataset.loc[:,'city_id'] = city_id
    temp_dataset.loc[:,'city_name'] = city_name
    
   # Nettoyer les champs de texte

    temp_dataset.loc[:, 'name'] = temp_dataset['name'].str.replace('\n', '')
    temp_dataset.loc[:, 'url'] = temp_dataset['url'].str.replace('\n', '')
    temp_dataset.loc[:, 'description'] = temp_dataset['description'].str.replace('\n', '')
    
   # Extraire la latitude et la longitude
    
    temp_dataset.loc[:,'coords'] = temp_dataset['coords'].str.split(',')
    temp_dataset.loc[:, 'longitude'] = temp_dataset['coords'].apply(lambda x : x[0])
    temp_dataset.loc[:, 'latitude'] = temp_dataset['coords'].apply(lambda x : x[1])
    
    # Suppression de la colonne coords (qui n'est plus utile)
    
    temp_dataset = temp_dataset.drop('coords', axis=1)
    
    # Ajouter au cadre de données de l'hôtel
    
    hotels = hotels.append(temp_dataset)

In [7]:
# Contrôle
hotels.head()

,city_id,city_name,hotel_id,name,url,latitude,longitude,score,description


In [ ]:
# Convert columns to convenient types
hotels.loc[:, 'score'] = hotels['score'].str.replace(',','.')
hotels = hotels.astype({'city_id': int,
                        'hotel_id': int,
                        'latitude': float,
                        'longitude': float,
                        'score': float})

In [ ]:
# Contrôle
hotels.head()

### 3. Enregistrez le DataFrame contenant toutes les informations sur les hôtels dans un fichier `.csv` :

In [ ]:
# Sauvegarder le DataFrame des hôtels dans un fichier .csv

hotels.to_csv('res/3_hotels.csv', index=False)

### 4. Facultatif : utilisez Plotly pour afficher tous les hôtels sur une carte :

In [ ]:
hotels_with_score = hotels.loc[hotels['score'].notnull(),:]

fig = px.scatter_mapbox(hotels_with_score, lat="latitude", lon="longitude", hover_name = 'name', zoom = 4,
                        hover_data = ['description'],
                        color = 'score', color_continuous_scale = 'thermal',
                        mapbox_style="carto-positron")
fig.show('iframe')

### 5. Utilisez `boto3` pour enregistrer les DataFrames sur les villes et les hôtels dans des fichiers `.csv` situés dans un nouveau seau S3 de votre compte AWS :

In [ ]:
# Initier une nouvelle session (si .aws/credentials n'existe pas)

# session = boto3.Session(aws_access_key_id="YOUR_ACCESS_KEY_ID", 
#                         aws_secret_access_key="YOUR_SECRET_ACCESS_KEY")


# Déclarez l'objet s3 et créez un nouveau seau(bucket)

s3 = boto3.resource("s3")  # s3 = session.resource("s3")
bucket_name = s3.create_bucket(Bucket="Kayak")

# Exporter les hôtels vers un fichier CSV et le télécharger.

hotels_csv = hotels.to_csv(index=False)
put_object = bucket_name.put_object(Key = "hotels.csv", Body = hotels_csv)

# Do the same for cities
cities_csv = cities.to_csv(index=False)
put_object = bucket_name.put_object(Key = "cities.csv", Body = cities_csv)